# Extract UMAP embeddings for each plate of CellProfiler features

NOTE: We are using the feature selected profiles per plate.

In [1]:
import glob
import pathlib
import pandas as pd
import numpy as np
import umap

from pycytominer.cyto_utils import infer_cp_features

## Generate Embeddings

### Set constant for whole plates

In [2]:
# Set constants
umap_random_seed = 0
umap_n_components = 2

# Set embeddings directory
output_dir = pathlib.Path("results")
output_dir.mkdir(parents=True, exist_ok=True)

### Set paths to all plates

In [3]:
# Set input path with single-cell profiles
data_dir = pathlib.Path("../3.preprocessing_features/data/single_cell_profiles")

# Select only the feature selected files
file_suffix = "*sc_feature_selected.parquet"

# Obtain file paths for all feature selected plates
fs_files = glob.glob(f"{data_dir}/{file_suffix}")
print(f"There are {len(fs_files)} feature selected files with the following paths:")
fs_files

There are 4 feature selected files with the following paths:


['../3.preprocessing_features/data/single_cell_profiles/localhost240928120001_sc_feature_selected.parquet',
 '../3.preprocessing_features/data/single_cell_profiles/localhost240927060001_sc_feature_selected.parquet',
 '../3.preprocessing_features/data/single_cell_profiles/localhost240927120001_sc_feature_selected.parquet',
 '../3.preprocessing_features/data/single_cell_profiles/localhost240926150001_sc_feature_selected.parquet']

### Generate dictionary with plate and data

In [4]:
# Load feature data into a dictionary, keyed on plate name without the suffix
cp_dfs = {x.split("/")[-1].split("_sc")[0]: pd.read_parquet(x) for x in fs_files}

# Print out useful information about each dataset
print(cp_dfs.keys())
[cp_dfs[x].shape for x in cp_dfs]

dict_keys(['localhost240928120001', 'localhost240927060001', 'localhost240927120001', 'localhost240926150001'])


[(12745, 641), (12397, 652), (12902, 684), (16566, 657)]

### Fit UMAP for whole plates

In [5]:
# Initialize a dictionary to store UMAP embeddings for each plate
umap_results = {}

# Fit UMAP features per dataset and save
for plate in cp_dfs:
    # Set output file for the UMAP
    output_umap_file = pathlib.Path(output_dir, f"UMAP_{plate}.parquet")

    # Check if the output file already exists
    if output_umap_file.exists():
        print(f"Skipping {output_umap_file.stem} as it already exists.")
        continue

    # Make sure to reinitialize UMAP instance per plate
    umap_fit = umap.UMAP(
        random_state=umap_random_seed, n_components=umap_n_components, n_jobs=1
    )

    # Set dataframe as the current plate
    cp_df = cp_dfs[plate]

    # Prior to running UMAP, fix Pathways bug for two compounds
    cp_df.loc[cp_df["Metadata_treatment"] == "UCD-0159258", "Metadata_Pathway"] = (
        "Angiogenesis"
    )
    cp_df.loc[cp_df["Metadata_treatment"] == "UCD-0001804", "Metadata_Pathway"] = "MAPK"

    # Process cp_df to separate features and metadata
    cp_features = infer_cp_features(cp_df)
    meta_features = infer_cp_features(cp_df, metadata=True)

    # Subset to only failing + DMSO and healthy + DMSO for fitting (avoid flooding from the compound treated cells)
    fit_subset = cp_df[
        (
            (cp_df["Metadata_cell_type"] == "healthy")
            & (cp_df["Metadata_treatment"] == "DMSO")
        )
        | (
            (cp_df["Metadata_cell_type"] == "failing")
            & (cp_df["Metadata_treatment"] == "DMSO")
        )
    ]

    # Fit on the subset
    umap_fit.fit(fit_subset.loc[:, cp_features])

    # Transform entire dataset and convert to pandas DataFrame
    embeddings = pd.DataFrame(
        umap_fit.transform(cp_df.loc[:, cp_features]),
        columns=[f"UMAP{x}" for x in range(0, umap_n_components)],
    )
    print(f"{embeddings.shape}: {plate}")

    # Combine with metadata
    cp_umap_with_metadata_df = pd.concat(
        [cp_df.loc[:, meta_features], embeddings], axis=1
    )

    # Add treatment type column
    cp_umap_with_metadata_df["Metadata_treatment_type"] = np.select(
        [
            (cp_umap_with_metadata_df["Metadata_cell_type"] == "healthy")
            & (cp_umap_with_metadata_df["Metadata_treatment"] == "DMSO"),
            (cp_umap_with_metadata_df["Metadata_cell_type"] == "failing")
            & (cp_umap_with_metadata_df["Metadata_treatment"] == "DMSO"),
            (cp_umap_with_metadata_df["Metadata_cell_type"] == "failing")
            & (cp_umap_with_metadata_df["Metadata_treatment"] != "DMSO"),
        ],
        ["healthy + DMSO", "failing + DMSO", "failing + compound"],
        default="other",
    )
    # Add new metadata column to the list
    meta_features.append("Metadata_treatment_type")

    # Check and adjust dtypes dynamically
    for col in cp_umap_with_metadata_df.columns:
        if col in meta_features:
            # Try converting to numeric first (if possible), if not, keep as string
            try:
                cp_umap_with_metadata_df[col] = pd.to_numeric(
                    cp_umap_with_metadata_df[col], errors="raise", downcast="integer"
                )
            except ValueError:
                # If can't convert to numeric, keep as string
                cp_umap_with_metadata_df[col] = cp_umap_with_metadata_df[col].astype(
                    str
                )
        else:
            # For UMAP embeddings, ensure they're float
            cp_umap_with_metadata_df[col] = cp_umap_with_metadata_df[col].astype(float)

        # Update the 'Pathway' column for failing + DMSO and healthy + DMSO
        cp_umap_with_metadata_df["Metadata_Pathway"] = cp_umap_with_metadata_df.apply(
            lambda row: (
                "failing + DMSO"
                if row["Metadata_cell_type"] == "failing"
                and row["Metadata_treatment"] == "DMSO"
                else (
                    "healthy + DMSO"
                    if row["Metadata_cell_type"] == "healthy"
                    and row["Metadata_treatment"] == "DMSO"
                    else row["Metadata_Pathway"]
                )
            ),
            axis=1,
        )

    # Store the UMAP result in the dictionary
    umap_results[plate] = cp_umap_with_metadata_df

    # Generate output file, drop unnamed column, and save
    cp_umap_with_metadata_df.to_parquet(output_umap_file, index=False)

(12745, 2): localhost240928120001
(12397, 2): localhost240927060001
(12902, 2): localhost240927120001
(16566, 2): localhost240926150001


## Combine all plates together to generate UMAP embeddings

In [6]:
# Get common features across all plates
common_columns = set.intersection(*[set(df.columns) for df in cp_dfs.values()])

# Use the first plate to identify metadata columns (those starting with 'Metadata_')
first_df = next(iter(cp_dfs.values()))
metadata_columns = [col for col in first_df.columns if col.startswith("Metadata_")]

# Get final columns to keep: intersection of common + metadata
final_columns = list(common_columns.union(metadata_columns))

# Subset each plate's dataframe to only those columns
cp_dfs_subset = {k: df[final_columns] for k, df in cp_dfs.items()}

# Combine all feature-selected plates with common features and metadata
combined_cp_df = pd.concat(cp_dfs_subset.values(), ignore_index=True)

# Verify shape and output
print(f"Combined shape: {combined_cp_df.shape}")
combined_cp_df.head()

Combined shape: (54610, 494)


,Cells_AreaShape_Zernike_3_1,Cytoplasm_Texture_Correlation_ER_3_01_256,Cells_RadialDistribution_RadialCV_ER_3of4,Nuclei_RadialDistribution_FracAtD_Mitochondria_1of4,Cells_AreaShape_Zernike_9_1,Nuclei_RadialDistribution_FracAtD_Actin_4of4,Nuclei_RadialDistribution_RadialCV_Mitochondria_1of4,Cells_Intensity_MaxIntensityEdge_Actin,Cytoplasm_Intensity_MaxIntensityEdge_Mitochondria,Nuclei_RadialDistribution_MeanFrac_PM_4of4,...,Cells_AreaShape_Compactness,Cells_AreaShape_Zernike_6_0,Nuclei_RadialDistribution_RadialCV_ER_2of4,Cytoplasm_Texture_InfoMeas2_ER_3_01_256,Nuclei_RadialDistribution_RadialCV_Mitochondria_2of4,Nuclei_Texture_Correlation_PM_3_00_256,Cells_Texture_InfoMeas2_ER_3_01_256,Cells_RadialDistribution_RadialCV_Hoechst_4of4,Cytoplasm_Correlation_Correlation_Mitochondria_PM,Cells_AreaShape_Zernike_9_3
0,0.807505,-0.739865,-0.936661,3.861680,-0.568185,0.343752,3.881613,0.717384,-1.247767,-1.791026,...,2.308930,-0.543098,-0.118815,-0.597940,0.108337,-0.038723,-0.830841,0.988169,-0.344781,0.024622
1,0.113661,-0.601948,-0.708422,-0.421215,0.659079,-0.059241,-0.511069,0.763051,-0.398439,-1.297398,...,0.993908,-0.953293,-0.565998,-0.801475,-0.355731,0.696999,-0.966826,-0.089889,-0.171927,0.219651
2,0.274435,-0.861723,-1.052734,-1.424283,-1.191864,-0.381104,-0.442557,0.601231,1.232831,-0.615197,...,1.282518,-0.990414,-0.725422,0.068916,3.598754,-0.569930,-0.032284,-0.431154,-0.388821,0.686977
3,-1.348966,0.398420,-0.843111,0.083470,-1.723041,0.129361,-0.668075,-0.687372,-0.772063,-1.750232,...,0.515245,1.859774,-0.287545,0.935916,-0.390530,1.555646,0.681503,1.025870,-0.258096,-1.111104
4,2.488497,-1.979927,-1.004789,5.840096,-0.160134,-2.533515,1.632098,1.845159,1.164695,-2.102925,...,-0.650072,-0.192049,1.531849,-0.959413,0.235294,1.003820,-1.295796,0.755141,0.153745,-1.306442


In [7]:
# Process combined_cp_df to separate features and metadata
combined_cp_features = infer_cp_features(combined_cp_df)
combined_meta_features = infer_cp_features(combined_cp_df, metadata=True)

# Subset to only failing + DMSO and healthy + DMSO for fitting
combined_fit_subset = combined_cp_df[
    (
        (combined_cp_df["Metadata_cell_type"] == "healthy")
        & (combined_cp_df["Metadata_treatment"] == "DMSO")
    )
    | (
        (combined_cp_df["Metadata_cell_type"] == "failing")
        & (combined_cp_df["Metadata_treatment"] == "DMSO")
    )
]

# Initialize UMAP instance
combined_umap_fit = umap.UMAP(
    random_state=umap_random_seed, n_components=umap_n_components, n_jobs=1
)

# Fit on the subset
combined_umap_fit.fit(combined_fit_subset.loc[:, combined_cp_features])

# Transform entire dataset and convert to pandas DataFrame
combined_embeddings = pd.DataFrame(
    combined_umap_fit.transform(combined_cp_df.loc[:, combined_cp_features]),
    columns=[f"UMAP{x}" for x in range(0, umap_n_components)],
)

# Combine with metadata
combined_cp_umap_with_metadata_df = pd.concat(
    [combined_cp_df.loc[:, combined_meta_features], combined_embeddings], axis=1
)

# Add treatment type column
combined_cp_umap_with_metadata_df["Metadata_treatment_type"] = np.select(
    [
        (combined_cp_umap_with_metadata_df["Metadata_cell_type"] == "healthy")
        & (combined_cp_umap_with_metadata_df["Metadata_treatment"] == "DMSO"),
        (combined_cp_umap_with_metadata_df["Metadata_cell_type"] == "failing")
        & (combined_cp_umap_with_metadata_df["Metadata_treatment"] == "DMSO"),
        (combined_cp_umap_with_metadata_df["Metadata_cell_type"] == "failing")
        & (combined_cp_umap_with_metadata_df["Metadata_treatment"] != "DMSO"),
    ],
    ["healthy + DMSO", "failing + DMSO", "failing + compound"],
    default="other",
)

# Update the 'Pathway' column for failing + DMSO and healthy + DMSO
combined_cp_umap_with_metadata_df["Metadata_Pathway"] = combined_cp_umap_with_metadata_df.apply(
    lambda row: (
        "failing + DMSO"
        if row["Metadata_cell_type"] == "failing"
        and row["Metadata_treatment"] == "DMSO"
        else (
            "healthy + DMSO"
            if row["Metadata_cell_type"] == "healthy"
            and row["Metadata_treatment"] == "DMSO"
            else row["Metadata_Pathway"]
        )
    ),
    axis=1,
)

# Save the combined UMAP embeddings to a file
combined_output_umap_file = pathlib.Path(output_dir, "UMAP_combined.parquet")
combined_cp_umap_with_metadata_df.to_parquet(combined_output_umap_file, index=False)

print(f"Combined UMAP embeddings saved to {combined_output_umap_file}")

Combined UMAP embeddings saved to results/UMAP_combined.parquet


In [8]:
combined_cp_umap_with_metadata_df.head()

,Metadata_Pathway,Metadata_treatment,Metadata_Cytoplasm_Parent_Cells,Metadata_Cells_Number_Object_Number,Metadata_cell_type,Metadata_Cytoplasm_Parent_Nuclei,Metadata_Image_Count_Cells,Metadata_Nuclei_Location_Center_Y,Metadata_Cells_Location_Center_X,Metadata_heart_number,...,Metadata_WellCol,Metadata_WellRow,Metadata_Plate,Metadata_Well,Metadata_Nuclei_Number_Object_Number,Metadata_heart_failure_type,Metadata_Nuclei_Location_Center_X,UMAP0,UMAP1,Metadata_treatment_type
0,healthy + DMSO,DMSO,1,1,healthy,6,9,277.977481,809.932735,7,...,2,B,localhost240928120001,B02,6,None,855.599578,14.039492,-2.672143,healthy + DMSO
1,healthy + DMSO,DMSO,1,1,healthy,3,4,209.544343,640.790427,7,...,2,B,localhost240928120001,B02,3,None,613.937003,12.391367,-2.804739,healthy + DMSO
2,healthy + DMSO,DMSO,1,1,healthy,3,9,280.023188,927.500461,7,...,2,B,localhost240928120001,B02,3,None,916.379710,12.981672,-2.519132,healthy + DMSO
3,healthy + DMSO,DMSO,1,1,healthy,3,18,80.037145,515.843583,7,...,2,B,localhost240928120001,B02,3,None,530.411492,12.088455,-3.910254,healthy + DMSO
4,healthy + DMSO,DMSO,1,1,healthy,4,10,69.265085,955.244723,7,...,2,B,localhost240928120001,B02,4,None,958.301695,11.054729,-2.796539,healthy + DMSO
